In [1]:
from circuit_finder.pretrained import load_mlp_transcoders

transcoder = load_mlp_transcoders([8])[8]
print(transcoder.cfg)


Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

LanguageModelSAERunnerConfig(model_name='gpt2-small', hook_point='blocks.8.ln2.hook_normalized', hook_point_layer=8, hook_point_head_index=None, dataset_path='Skylion007/openwebtext', is_dataset_tokenized=False, context_size=128, use_cached_activations=False, cached_activations_path='activations/Skylion007_openwebtext/gpt2-small/blocks.8.ln2.hook_normalized', d_in=768, n_batches_in_buffer=128, total_training_tokens=300000000, store_batch_size=32, device='cuda', seed=42, dtype=torch.float32, is_transcoder=True, out_hook_point='blocks.8.hook_mlp_out', out_hook_point_layer=8, d_out=768, is_sparse_connection=False, sparse_connection_sae_path=None, sparse_connection_l1_coeff=None, sparse_connection_use_W_enc=True, b_dec_init_method='mean', expansion_factor=32, from_pretrained_path=None, l1_coefficient=8e-05, lr=0.0004, lr_scheduler_name='constantwithwarmup', lr_warm_up_steps=5000, train_batch_size=4096, use_ghost_grads=True, feature_sampling_window=1000, feature_sampling_method='Anthropic',

In [3]:
from dataclasses import asdict
for key, value in asdict(transcoder.cfg).items():
    print(f'{key}: {value}')

model_name: gpt2-small
hook_point: blocks.8.ln2.hook_normalized
hook_point_layer: 8
hook_point_head_index: None
dataset_path: Skylion007/openwebtext
is_dataset_tokenized: False
context_size: 128
use_cached_activations: False
cached_activations_path: activations/Skylion007_openwebtext/gpt2-small/blocks.8.ln2.hook_normalized
d_in: 768
n_batches_in_buffer: 128
total_training_tokens: 300000000
store_batch_size: 32
device: cuda
seed: 42
dtype: torch.float32
is_transcoder: True
out_hook_point: blocks.8.hook_mlp_out
out_hook_point_layer: 8
d_out: 768
is_sparse_connection: False
sparse_connection_sae_path: None
sparse_connection_l1_coeff: None
sparse_connection_use_W_enc: True
b_dec_init_method: mean
expansion_factor: 32
from_pretrained_path: None
l1_coefficient: 8e-05
lr: 0.0004
lr_scheduler_name: constantwithwarmup
lr_warm_up_steps: 5000
train_batch_size: 4096
use_ghost_grads: True
feature_sampling_window: 1000
feature_sampling_method: Anthropic
resample_batches: 32
feature_reinit_scale: 0.2

In [ ]:
from circuit_finder.pretrained import load_model

model = load_model()
print(model)

In [4]:
from circuit_finder.patching.transcoder_replacement_ctx import (
    TranscoderReplacementContext,
)

text = "When John and Mary went to the shops, John gave a bottle to Mary"

orig_loss = model.run_with_cache(text, return_type="loss")
print(orig_loss)

with TranscoderReplacementContext(model, [transcoder]):
    loss, _ = model.run_with_cache(text, return_type="loss")
    print(loss)

(tensor(3.7746, device='cuda:0'), ActivationCache with keys ['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.hook_attn_in', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.attn.hook_result', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.hook_mlp_in', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.hook_attn_in', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.attn.hook_result', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.hook

In [5]:
transcoders = load_mlp_transcoders()
with TranscoderReplacementContext(model, list(transcoders.values())):
    loss, _ = model.run_with_cache(text, return_type="loss")
    print(loss)

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

tensor(4.1620, device='cuda:0', grad_fn=<NegBackward0>)
